# Financial Document Analysis with LlamaIndex

In this example notebook, we showcase how to perform financial analysis over [**10-K**](https://en.wikipedia.org/wiki/Form_10-K) documents with the [**LlamaIndex**](https://gpt-index.readthedocs.io/en/latest/) framework with just a few lines of code.

## Notebook Outline
* [Introduction](#Introduction)
* [Setup](#Setup)
* [Data Loading & Indexing](#Data-Loading-and-Indexing)
* [Simple QA](#Simple-QA)
* [Advanced QA - Compare and Contrast](#Advanced-QA---Compare-and-Contrast)


## Introduction

### LLamaIndex
[LlamaIndex](https://gpt-index.readthedocs.io/en/latest/) is a data framework for LLM applications.
You can get started with just a few lines of code and build a retrieval-augmented generation (RAG) system in minutes.
For more advanced users, LlamaIndex offers a rich toolkit for ingesting and indexing your data, modules for retrieval and re-ranking, and composable components for building custom query engines.

See [full documentation](https://gpt-index.readthedocs.io/en/latest/) for more details.

### Financial Analysis over 10-K documents
A key part of a financial analyst's job is to extract information and synthesize insight from long financial documents.
A great example is the 10-K form - an annual report required by the U.S. Securities and Exchange Commission (SEC), that gives a comprehensive summary of a company's financial performance.
These documents typically run hundred of pages in length, and contain domain-specific terminology that makes it challenging for a layperson to digest quickly.


We showcase how LlamaIndex can support a financial analyst in quickly extracting information and synthesize insights **across multiple documents** with very little coding. 

## Setup

To begin, we need to install the llama-index library

In [ ]:
!pip install llama-index==0.12.8 pypdf

Now, we import all modules used in this tutorial

In [ ]:
import os
from llama_index.llms.openai import OpenAI
from llama_index.core.readers import SimpleDirectoryReader
from llama_index.core.indices import VectorStoreIndex
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.settings import Settings
from llama_index.embeddings.openai.base import OpenAIEmbedding

Before we start, we can configure the LLM provider and model that will power our RAG system.  
Here, we pick `gpt-4o-mini` from OpenAI.  

In [ ]:
os.environ['OPENAI_API_KEY'] = 'YOUR OPENAI API KEY'
llm = OpenAI(temperature=0, model_name="gpt-4o-mini")

Set the LLM to the previously configured OpenAI model and the embedding model to the OpenAI embedding model using `Settings`.

In [ ]:
Settings.llm = llm
Settings.embed_model = OpenAIEmbedding()
Settings.chunk_size = 1024

## Data Loading and Indexing

Now, we load and parse 2 PDFs (one for Uber 10-K in 2021 and another for Lyft 10-k in 2021).    
Under the hood, the PDFs are converted to plain text `Document` objects, separate by page.  

> Note: this operation might take a while to run, since each document is more than 100 pages.

In [ ]:
lyft_docs = SimpleDirectoryReader(input_files=["../data/10k/lyft_2021.pdf"]).load_data()
uber_docs = SimpleDirectoryReader(input_files=["../data/10k/uber_2021.pdf"]).load_data()

In [ ]:
print(f'Loaded lyft 10-K with {len(lyft_docs)} pages')
print(f'Loaded Uber 10-K with {len(uber_docs)} pages')

Loaded lyft 10-K with 238 pages
Loaded Uber 10-K with 307 pages


Now, we can build an (in-memory) `VectorStoreIndex` over the documents that we've loaded.  

> Note: this operation might take a while to run, since it calls OpenAI API for computing vector embedding over document chunks.

In [ ]:
lyft_index = VectorStoreIndex.from_documents(lyft_docs)
uber_index = VectorStoreIndex.from_documents(uber_docs)

## Simple QA

Now we are ready to run some queries against our indices!  
To do so, we first configure a `QueryEngine`, which just captures a set of configurations for how we want to query the underlying index.

For a `VectorStoreIndex`, the most common configuration to adjust is `similarity_top_k` which controls how many document chunks (which we call `Node` objects) are retrieved to use as context for answering our question.

In [ ]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)

In [ ]:
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

Let's see some queries in action!

In [ ]:
response = await lyft_engine.aquery('What is the revenue of Lyft in 2021? Answer in millions with page reference')

In [ ]:
print(response)

The revenue of Lyft in 2021 was $3.21 billion. (Page reference: 79)


In [ ]:
response = await uber_engine.aquery('What is the revenue of Uber in 2021? Answer in millions, with page reference')

In [ ]:
print(response)

The revenue of Uber in 2021 was $17,455 million. (Page reference: 98)


## Advanced QA - Compare and Contrast

For more complex financial analysis, one often needs to reference multiple documents.  

As a example, let's take a look at how to do compare-and-contrast queries over both Lyft and Uber financials.  
For this, we build a `SubQuestionQueryEngine`, which breaks down a complex compare-and-contrast query, into simpler sub-questions to execute on respective sub query engine backed by individual indices.

In [ ]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine, 
        metadata=ToolMetadata(name='lyft_10k', description='Provides information about Lyft financials for year 2021')
    ),
    QueryEngineTool(
        query_engine=uber_engine, 
        metadata=ToolMetadata(name='uber_10k', description='Provides information about Uber financials for year 2021')
    ),
]

s_engine = SubQuestionQueryEngine.from_defaults(query_engine_tools=query_engine_tools)

Let's see these queries in action!

In [ ]:
response = await s_engine.aquery('Compare and contrast the customer segments and geographies that grew the fastest')

Generated 4 sub questions.
[lyft_10k] Q: What were the customer segments that grew the fastest for Lyft in 2021?
[lyft_10k] Q: What were the geographies that grew the fastest for Lyft in 2021?
[uber_10k] Q: What were the customer segments that grew the fastest for Uber in 2021?
[uber_10k] Q: What were the geographies that grew the fastest for Uber in 2021?
[uber_10k] A: Chicago, Miami, New York City in the United States, Sao Paulo in Brazil, and London in the United Kingdom were the geographies that grew the fastest for Uber in 2021.
[uber_10k] A: The customer segments that grew the fastest for Uber in 2021 were the membership programs, specifically Uber One, Uber Pass, Eats Pass, and Rides Pass.
[lyft_10k] A: The geographies that grew the fastest for Lyft in 2021 were those where vaccines were more widely distributed and communities fully reopened, resulting in a 36% increase in revenue compared to the prior year. Additionally, the number of Active Riders increased by 49.2% in the fou

In [ ]:
print(response)

The customer segments that grew the fastest for Lyft in 2021 were the Active Riders, with a notable increase of 49.2% in the fourth quarter of 2021 compared to the same period in 2020. On the other hand, Uber experienced the fastest growth in membership programs such as Uber One, Uber Pass, Eats Pass, and Rides Pass.

In terms of geographies, Lyft saw the fastest growth in areas where vaccines were widely distributed and communities fully reopened, leading to a 36% increase in revenue compared to the previous year. For Uber, the fastest-growing geographies in 2021 were Chicago, Miami, New York City in the United States, Sao Paulo in Brazil, and London in the United Kingdom.


In [ ]:
response = await s_engine.aquery('Compare revenue growth of Uber and Lyft from 2020 to 2021')

Generated 4 sub questions.
[uber_10k] Q: What was the revenue of Uber in 2020?
[uber_10k] Q: What was the revenue of Uber in 2021?
[lyft_10k] Q: What was the revenue of Lyft in 2020?
[lyft_10k] Q: What was the revenue of Lyft in 2021?
[uber_10k] A: $17,455
[uber_10k] A: The revenue of Uber in 2020 was $11,139 million.
[lyft_10k] A: The revenue of Lyft in 2021 was $3,208,323,000.
[lyft_10k] A: Lyft's revenue in 2020 was $2,364,681.


In [ ]:
print(response)

The revenue growth of Uber from 2020 to 2021 was $6,316 million, while the revenue growth of Lyft during the same period was $843,642,000.
